Read gene info(hg38) from a file, for each sample, count the overlaps in each gene than save to db.  
This is the modified version based on the cytoband per sample. Gene list from biomart
Current version.

In [2]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
import copy

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

## Script Verison  
Test data

In [ ]:
db = MongoClient()['Rebased']['skin']
samples = []
for sam in db.find():
    samples.append(sam)

Parse genes manually to a list of lists for faster computation later

In [3]:
# genes is a list of 24 lists, represents 24 chromosomes. Each sub-list stores the genes of that chromosome as another list
genes = []
for i in range(24):
    genes.append([])
    

with open('/Users/bogao/DataFiles/Data/genome/protein_genes_biomart.tsv','r') as fin:
    next(fin)
    for line in fin:
        tokens = line.strip().split('\t')
        if ('_' in tokens[2]) or ('M' in tokens[2]):
            continue
        elif 'X' in tokens[2]:
            chro = 22
        elif 'Y' in tokens[2]:
            chro = 23
        else:
            chro = int(tokens[2])-1
        
        if len(line) == 6:
            symbol = tokens[5]
        else:
            symbol = ''
        # index of counters: total_dup=4, total_del=5, dup_length=6, del_length=7, dup_count=8, del_count=9,   
        # list was used in the dataset version, replace with a dict here
        # info = [int(tokens[1]),int(tokens[2]), *tokens[3:], *[0]*6] 
        info = {'start':int(tokens[3]), 'end':int(tokens[4]), 'id':tokens[0], 'symbol':symbol, 
                'total_dup':0, 'total_del':0, 'dup_length':0, 'del_length':0, 'dup_count':0, 'del_count':0}
        genes[chro].append(info)

        
    

Most of the time, a gene is only cover once, just for strange cases. Codes adopted from dataset aggregation.

In [ ]:
c = 1

for sam in samples:
    this_genes = copy.deepcopy(genes)
    
    for seg in sam['segments']:
        if seg['chro'] == 'X':
            chro = 22
        elif seg['chro'] == 'Y':
            chro = 23
        else:
            chro = int(seg['chro'])-1
            
        # find all matching bands
        for b in this_genes[chro]:
            if (b['start'] < seg['end']) and (b['end'] >= seg['start']):
                size = min(b['end'],seg['end']) - max(b['start'],seg['start'])
                
                # dup
                if seg['value'] > 0:
                    # update total value
                    b['total_dup'] += size*seg['value']
                    # update total_length
                    b['dup_length'] += size
                    # update count
                    b['dup_count'] +=1
                # del
                else:
                    # update total value
                    b['total_del'] += size*seg['value']
                    # update total_length
                    b['del_length'] += size
                    # update count
                    b['del_count'] +=1
        
        # flat the nested bands
        flat_genes = []
        idx = 1
        for ch in this_genes:
            if idx == 23:
                chro = 'X'
            elif idx == 24:
                chro = 'Y'
            else:
                chro = str(idx)
            for b in ch:
                # add chromosom
                b['chro'] = chro
                # compute average signals
                if b['dup_length'] >0:
                    b['ave_dup'] = b['total_dup'] / b['dup_length']
                else:
                    b['ave_dup'] = 0
                    
                if b['del_length'] >0:
                    b['ave_del'] = b['total_del'] / b['del_length']
                else:
                    b['ave_del'] = 0
                
#                 # psudo-probes
#                 b['probes'] = 10
#                 b['value'] = b['ave_del'] + b['ave_dup']
                flat_genes.append(b)
            idx +=1        
    sam['genes'] = flat_genes
    print(c, end='\r')
    c +=1
    
    if c>10:
        break

## Funtion version

In [5]:
def geneOverlap(genes, dbin, vtype='normalized'):

    c = 1
    for sam in dbin.find():
        this_genes = copy.deepcopy(genes)
        if (vtype in sam.keys()) and (sam[vtype] != None):
            for seg in sam[vtype]:
                if seg['probes'] >4:
                    
                    if seg['value'] >2:
                        val = 2
                    else:
                        val = seg['value']
                    
                    if seg['chro'] == 'X':
                        chro = 22
                    elif seg['chro'] == 'Y':
                        chro = 23
                    else:
                        chro = int(seg['chro'])-1

                    # find all matching bands
                    for b in this_genes[chro]:
                        if (b['start'] < seg['end']) and (b['end'] >= seg['start']):
                            size = min(b['end'],seg['end']) - max(b['start'],seg['start'])

                            # dup
                            if seg['value'] > 0:
                                # update total value
                                b['total_dup'] += size*seg['value']
                                # update total_length
                                b['dup_length'] += size
                                # update count
                                b['dup_count'] +=1
                            # del
                            else:
                                # update total value
                                b['total_del'] += size*seg['value']
                                # update total_length
                                b['del_length'] += size
                                # update count
                                b['del_count'] +=1

                # flat the nested bands
                flat_genes = []
                idx = 1
                for ch in this_genes:
                    if idx == 23:
                        chro = 'X'
                    elif idx == 24:
                        chro = 'Y'
                    else:
                        chro = str(idx)
                    for b in ch:
                        # add chromosom
                        b['chro'] = chro
                        # compute average signals
                        if b['dup_length'] >0:
                            b['ave_dup'] = b['total_dup'] / b['dup_length']
                        else:
                            b['ave_dup'] = 0

                        if b['del_length'] >0:
                            b['ave_del'] = b['total_del'] / b['del_length']
                        else:
                            b['ave_del'] = 0

                        flat_genes.append(b)
                    idx +=1        
            dbin.update_one({'sample_id':sam['sample_id']}, {'$set':{'genes':flat_genes}}, upsert=True)
            print(c, end='\r')
            c +=1
    return 0

In [6]:
# ret = geneOverlap(genes, MongoClient()['Rebased']['skin'])
# ret = geneOverlap(genes, MongoClient()['Rebased']['ovary'])
ret = geneOverlap(genes, MongoClient()['Rebased']['breast'])

## Testings

In [ ]:
import sys, os
sys.path.append('/Users/bogao/Desktop/projects/Relative copy number/Python/mecan')
import mecan4cna.algorithms as alg
import mecan4cna.common as comm

In [ ]:
db = MongoClient()['Rebased']['skin']
test_genes = []
for sam in db.find():
    test_genes.append(sam)

In [ ]:
idx = 20
comm.plotSegments(test_genes[idx]['segments'])

In [ ]:
for seg in test_genes[idx]['genes']:
    seg['probes'] = 10
    seg['value'] = seg['ave_del'] + seg['ave_dup']

comm.plotSegments(test_genes[idx]['genes'])

In [ ]:
samples[2]['cytobands'][22]